In [3]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import random
import sklearn.metrics as metrics
import json
import pretty_midi
import librosa

# import transformers and matplotlib in the cell when you call them, not globally

In [5]:
json_path = '/home/hice1/sgoel83/scratch/Maestro/maestro-v3.0.0/maestro-v3.0.0.json'
data_path = '/home/hice1/sgoel83/scratch/Maestro/maestro-v3.0.0/'

In [6]:
def load_train_data(json_path):
    with open(json_path, 'r') as file:
        train_data = json.load(file)
    return train_data

# train_data = load_train_data(json_path)ces once pickle file is complete 
# indices_2018 = [index for index, year in train_data.items() if year == 2018]

# Audio 2018 extraction 

In [15]:
audio2018_file_names = []
csv_maestro_wav = pd.read_csv(data_path+'maestro-v3.0.0.csv')
for idx, audio_file_a in enumerate(csv_maestro_wav['audio_filename']):
    if audio_file_a[:4] == '2018':
        audio2018_file_names.append((idx, audio_file_a))
print(audio2018_file_names)
print(len(audio2018_file_names))

[(0, '2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav'), (18, '2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--4.wav'), (19, '2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--2.wav'), (21, '2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--2.wav'), (22, '2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_11_R1_2018_wav--2.wav'), (23, '2018/MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--2.wav'), (26, '2018/MIDI-Unprocessed_Recital12_MID--AUDIO_12_R1_2018_wav--2.wav'), (35, '2018/MIDI-Unprocessed_Recital13-15_MID--AUDIO_15_R1_2018_wav--1.wav'), (61, '2018/MIDI-Unprocessed_Recital20_MID--AUDIO_20_R1_2018_wav--3.wav'), (69, '2018/MIDI-Unprocessed_Recital1-3_MID--AUDIO_02_R1_2018_wav--4.wav'), (74, '2018/MIDI-Unprocessed_Recital16_MID--AUDIO_16_R1_2018_wav--3.wav'), (75, '2018/MIDI-Unprocessed_Recital9-11_MID--AUDIO_10_R1_2018_wav--3.wav'), (87, '2018/MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--2.wav'), (88, '2018/MIDI-Unprocessed_

In [22]:
idx_to_cqt = {}

In [23]:
for (idx, audio_file_name) in audio2018_file_names:

    audio_data = data_path + audio_file_name
    print(f"Converting audio file from {audio_file_name.split('/')[0]}, with index {idx}, and file name {audio_file_name.split('/')[1]}")
    x , sr = librosa.load(audio_data)

    librosa.load(audio_data, sr=44100)

    X = librosa.cqt(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    idx_to_cqt[idx] = Xdb

    with open('/home/hice1/sgoel83/scratch/wav_pickle/cqt2018.pickle', 'wb+' if idx == 0 else 'ab+') as handle:
        pickle.dump({idx: Xdb}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Converting audio file from 2018, with index 0, and file name MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav
Converting audio file from 2018, with index 18, and file name MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--4.wav
Converting audio file from 2018, with index 19, and file name MIDI-Unprocessed_Recital17-19_MID--AUDIO_18_R1_2018_wav--2.wav
Converting audio file from 2018, with index 21, and file name MIDI-Unprocessed_Recital5-7_MID--AUDIO_07_R1_2018_wav--2.wav
Converting audio file from 2018, with index 22, and file name MIDI-Unprocessed_Recital9-11_MID--AUDIO_11_R1_2018_wav--2.wav
Converting audio file from 2018, with index 23, and file name MIDI-Unprocessed_Chamber6_MID--AUDIO_20_R3_2018_wav--2.wav
Converting audio file from 2018, with index 26, and file name MIDI-Unprocessed_Recital12_MID--AUDIO_12_R1_2018_wav--2.wav
Converting audio file from 2018, with index 35, and file name MIDI-Unprocessed_Recital13-15_MID--AUDIO_15_R1_2018_wav--1.wav
Converting audio 

In [25]:
print(idx_to_cqt.keys())
print(len(idx_to_cqt.keys()))

dict_keys([0, 18, 19, 21, 22, 23, 26, 35, 61, 69, 74, 75, 87, 88, 89, 97, 110, 112, 114, 187, 223, 234, 255, 257, 280, 317, 347, 377, 392, 393, 413, 414, 443, 444, 445, 446, 453, 454, 455, 456, 476, 479, 672, 678, 679, 691, 711, 717, 724, 729, 778, 782, 852, 873, 888, 920, 931, 934, 936, 939, 945, 972, 1003, 1009, 1014, 1015, 1017, 1022, 1031, 1051, 1054, 1094, 1095, 1106, 1109, 1110, 1138, 1141, 1142, 1144, 1159, 1165, 1166, 1181, 1194, 1199, 1200, 1216, 1224, 1240, 1253, 1255, 1274])
93


In [32]:
# # pip install this
# import gdown

# url = "https://drive.google.com/uc?id=1nxAI7CWnVug1gzj6vj8PUbvHVsIK0Y1Q"
# output = "<path/to/file/file-name>.pickle"
# gdown.download(url, output)

# MIDI2018 and CQT2018 final dicts

In [33]:
# MIDI Pickle file

import pickle
midi_final_dict = {}
with open('/home/hice1/sgoel83/scratch/midi_pickle/midi2018.pickle', 'rb') as f:
    while True:
        try:
            data = pickle.load(f)
            midi_final_dict = {**midi_final_dict, **data}
        except EOFError:
            break

In [34]:
print(len(midi_final_dict.keys()))
print(midi_final_dict.keys())

93
dict_keys([0, 18, 19, 21, 22, 23, 26, 35, 61, 69, 74, 75, 87, 88, 89, 97, 110, 112, 114, 187, 223, 234, 255, 257, 280, 317, 347, 377, 392, 393, 413, 414, 443, 444, 445, 446, 453, 454, 455, 456, 476, 479, 672, 678, 679, 691, 711, 717, 724, 729, 778, 782, 852, 873, 888, 920, 931, 934, 936, 939, 945, 972, 1003, 1009, 1014, 1015, 1017, 1022, 1031, 1051, 1054, 1094, 1095, 1106, 1109, 1110, 1138, 1141, 1142, 1144, 1159, 1165, 1166, 1181, 1194, 1199, 1200, 1216, 1224, 1240, 1253, 1255, 1274])


In [35]:
# CQT WAV Pickle file

import pickle
cqt_final_dict = {}
with open('/home/hice1/sgoel83/scratch/wav_pickle/cqt2018.pickle', 'rb') as f:
    while True:
        try:
            cqt_data = pickle.load(f)
            cqt_final_dict = {**cqt_final_dict, **cqt_data}
        except EOFError:
            break

In [36]:
print(len(cqt_final_dict.keys()))
print(cqt_final_dict.keys())

93
dict_keys([0, 18, 19, 21, 22, 23, 26, 35, 61, 69, 74, 75, 87, 88, 89, 97, 110, 112, 114, 187, 223, 234, 255, 257, 280, 317, 347, 377, 392, 393, 413, 414, 443, 444, 445, 446, 453, 454, 455, 456, 476, 479, 672, 678, 679, 691, 711, 717, 724, 729, 778, 782, 852, 873, 888, 920, 931, 934, 936, 939, 945, 972, 1003, 1009, 1014, 1015, 1017, 1022, 1031, 1051, 1054, 1094, 1095, 1106, 1109, 1110, 1138, 1141, 1142, 1144, 1159, 1165, 1166, 1181, 1194, 1199, 1200, 1216, 1224, 1240, 1253, 1255, 1274])


# Shaping into x and y

In [26]:
# Shaping parameters
batch_size = 32
time_slices = 350
frequency_bins_x = 288
duration_x = 5
channels_x = 1
keys_y = 88

In [10]:
x_parameters = {
    'batch_size': batch_size,
    'time_slices': time_slices,
    'frequency_bins': frequency_bins_x,
    'duration': duration_x,
    'channels': channels_x}

y_parameters = {
    'batch_size': batch_size,
    'time_slices': time_slices,
    'num_keys': keys_y
}

In [51]:
# x : (84, 32, 360, 5, 1)
# y : (88, 32, 360)

x = []
count = 0
count_0 = 0
max_ret = 0
min_ret = float("inf")
for idx, audio in cqt_final_dict.items():
    # print("Shape 0:" , audio.shape[0])
    # print("Shape 1:" , audio.shape[1])
    count += audio.shape[1]
    count_0 += audio.shape[0]
    # print(audio.shape[1]*audio.shape[0])
    max_ret = max(max_ret, audio.shape[1])
    min_ret = min(min_ret, audio.shape[1])
print("min_ret", min_ret)
print("max_ret", max_ret)
print(count/93)
print(count_0/93)

min_ret 6114
max_ret 110392
45800.76344086022
84.0


In [70]:
import numpy as np

# Target shape constants
target_shape = (84, 32, 360, 5, 1)
required_elements = np.prod(target_shape)

# Initialize a new dictionary to store the reshaped arrays
reshaped_cqt_dict = {}

for key, spectrogram in cqt_final_dict.items():
    # Get the current shape of the spectrogram
    current_shape = spectrogram.shape
    num_elements = current_shape[0] * current_shape[1]

    # Flatten the array to a 1D vector
    flat_spectrogram = spectrogram.flatten()

    # Check if padding or truncation is needed
    if num_elements < required_elements:
        # Pad with zeros if the spectrogram has fewer elements
        flat_spectrogram = np.pad(flat_spectrogram, (0, required_elements - num_elements), 'constant')
    elif num_elements > required_elements:
        # Truncate if there are more elements than needed
        flat_spectrogram = flat_spectrogram[:required_elements]

    # Reshape to the target shape
    reshaped_spectrogram = flat_spectrogram.reshape(target_shape)

    # Store in the new dictionary
    reshaped_cqt_dict[key] = reshaped_spectrogram

# Now reshaped_cqt_dict contains all spectrograms in the desired shape

In [69]:
for idx, value in reshaped_cqt_dict.items():
    first = value
    break
print(first.shape)

for orgid, org_val in cqt_final_dict.items():
    org_first = org_val
    break
print(org_first.shape)

orgv = org_first[:, :32]
fir = first[:, 0, :, 0, 0]
print(orgv.shape)
print(fir.shape)
print(fir.T==orgv)

(32, 360, 84, 5, 1)
(84, 30331)
(84, 32)
(32, 84)
[[ True False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [39]:
# x : (84, 32, 360, 5, 1)
# y : (84, 32, 360)

y = []
count = 0
count_0 = 0
max_ret = 0
min_ret = float("inf")

for idx, midi in midi_final_dict.items():
    # print("Shape 0:" , midi.shape[0])
    # print("Shape 1:" , midi.shape[1])
    count += midi.shape[1]
    count_0 += midi.shape[0]
    # print(midi.shape[1]*midi.shape[0])
    max_ret = max(max_ret, midi.shape[1])
    min_ret = min(min_ret, midi.shape[1])
print("min_ret", min_ret)
print("max_ret", max_ret)
print(count/93)
print(count_0/93)

min_ret 14140
max_ret 256228
106282.04301075269
128.0


In [ ]:
# A0 is 21, C8 is 108

In [ ]:
import numpy as np

# Target shape constants
target_shape = (88, 32, 360)
required_elements = np.prod(target_shape)

# Initialize a new dictionary to store the reshaped arrays
reshaped_midi_dict = {}

for key, midi_array in midi_final_dict.items():
    # Select rows from 21 to 108 (inclusive)
    piano_segment = midi_array[21:109, :]

    # Flatten the selected segment
    flat_segment = piano_segment.flatten()
    num_elements = flat_segment.size

    # Check if padding or truncation is needed
    if num_elements < required_elements:
        # Pad with zeros if there are fewer elements
        flat_segment = np.pad(flat_segment, (0, required_elements - num_elements), 'constant')
    elif num_elements > required_elements:
        # Truncate if there are more elements
        flat_segment = flat_segment[:required_elements]

    # Reshape to the target shape
    reshaped_segment = flat_segment.reshape(target_shape)

    # Store in the new dictionary
    reshaped_midi_dict[key] = reshaped_segment

# Now reshaped_midi_dict contains all MIDI arrays in the desired shape

# Hyperparameters and Training

In [9]:
# Hyperparameters for the model
